# Identify a video for the March PI Meeting Demo

In [1]:
import os
import json
import simplejson

import pandas as pd

from charm.data import utils

In [2]:
meta_filepath = '/home/iron-man/Documents/data/charm/transformed/metadata.csv'
meta_df = pd.read_csv(meta_filepath)

In [3]:
meta_df

/home/iron-man/Documents/charm/.venv/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/iron-man/Documents/charm/.venv/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,release,catalog_id,file_uid,url,modality,start,end,transcribed,utterance_count,emotion_count,...,unwrapped_md5,download_date,content_date,status_in_corpus,legacy_catalog_id,original_file_id,type,file_path,length,version
0,R1,LDC2022E11,M010009A4,na,video,NaN,NaN,True,45.0,NaN,...,1c797fd8bd832fe4c7244a7b9b0aa2a7,na,na,present,LDC2015R18,VVC008300,NaN,NaN,NaN,V1.0
1,R1,LDC2022E11,M010009BC,na,video,0.0,300.0,True,90.0,3.0,...,4b121a497725dcf1f232d00c119bd823,na,na,present,LDC2015R18,VVC016445,NaN,NaN,NaN,V1.0
2,R1,LDC2022E11,M010009BE,na,video,NaN,NaN,True,67.0,NaN,...,83d42079d22abe1f576be92afc182474,na,na,present,LDC2015R18,VVC020973,NaN,NaN,NaN,V1.0
3,R1,LDC2022E11,M010009CZ,na,video,NaN,NaN,True,89.0,NaN,...,07843653514dad5894b5782fe1475ee4,na,na,present,LDC2015R18,VVC011554,NaN,NaN,NaN,V1.0
4,R1,LDC2022E11,M010009D0,na,video,NaN,NaN,True,29.0,NaN,...,f0a8d0563a8d6480675f21df6c0acbd2,na,na,present,LDC2015R18,VVC011561,NaN,NaN,NaN,V1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16193,Mini-Eval,LDC2022E22,M01000HUA,na,text,0.0,678.0,NaN,NaN,NaN,...,72c4da8e36c41fc1eef2e0ebcbd19da3,na,na,present,na,na,text,data/text/psm/M01000HUA.psm.xml,681.0,V1.0
16194,Mini-Eval,LDC2022E22,M01000HUF,na,text,NaN,NaN,NaN,NaN,NaN,...,c2e8be27766da87f47d1a234e64bb7eb,na,na,present,na,na,text,data/text/psm/M01000HUF.psm.xml,982.0,V1.0
16195,Mini-Eval,LDC2022E22,M01000HUK,na,text,NaN,NaN,NaN,NaN,NaN,...,f226937cf10a0233c14bd3142f561c86,na,na,present,na,na,text,data/text/psm/M01000HUK.psm.xml,1224.0,V1.0
16196,Mini-Eval,LDC2022E22,M01000HUP,na,text,517.0,1129.0,NaN,NaN,NaN,...,33b9d19215c93ca6983dc1b6d2800754,na,na,present,na,na,text,data/text/psm/M01000HUP.psm.xml,1688.0,V1.0


In [4]:
# subset to mini-eval videos
file_filter = (meta_df['release'] == 'Mini-Eval') & (meta_df['modality'] == 'video')
df = meta_df[file_filter]

In [5]:
# find a file that's been annotated with all tasks
df = df[df[['valence_arousal_count', 'norms_count', 'changepoint_count', 'emotions_count']].notnull().all(axis=1)]

In [6]:
# TODO: load annotations and find a video with a reasonable number of annotations

In [7]:
# 7
video = df.sort_values(by=['utterance_count'], ascending=[True]).iloc[8]

In [8]:
video['file_uid']

'M010040RH'

In [9]:
with open(f'{video["file_uid"]}_metadata.json', 'w') as f:
    simplejson.dump(video.to_dict(), f, ignore_nan=True)

In [10]:
# strip LDC header
# translate with whisper

In [11]:
# load annotations for this file
result = utils.load_ldc_annotation('/home/iron-man/Documents/data/charm/raw/LDC2023E01_CCU_TA1_Mandarin_Chinese_Mini_Evaluation_Annotation_Unsequestered/')

In [12]:
anno_dfs, segment_df, version_df = result

In [13]:
anno_dfs.keys()

dict_keys(['valence_arousal.tab', 'changepoint.tab', 'norms.tab', 'emotions.tab'])

## Valence and Arousal

In [14]:
valence_df = anno_dfs['valence_arousal.tab']

In [15]:
# filter out noann rows
valence_df = valence_df[valence_df['valence_continuous'] != 'noann'].reset_index(drop=True)

In [16]:
# convert cols to floats
valence_convert_cols = ['valence_continuous', 'valence_binned', 'arousal_continuous', 'arousal_binned']
valence_df.loc[:, valence_convert_cols] = valence_df[valence_convert_cols].astype(float)

/tmp/ipykernel_4015467/3133681194.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  valence_df.loc[:, valence_convert_cols] = valence_df[valence_convert_cols].astype(float)


In [17]:
# drop the user_id column
valence_df.drop(columns=['user_id'], inplace=True)

In [18]:
# average valence over annotators
valence_df = valence_df.groupby(['file_id', 'segment_id'], as_index=False).mean()

In [19]:
valence_df = valence_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

In [20]:
valence_df = valence_df[valence_df['file_id'] == video['file_uid']]

In [21]:
valence_df

,file_id,segment_id,valence_continuous,valence_binned,arousal_continuous,arousal_binned,start,end
23214,M010040RH,M010040RH_0001,622.000000,3.666667,341.333333,2.000000,37.9,52.9
23215,M010040RH,M010040RH_0002,756.666667,4.333333,548.666667,3.000000,52.9,67.9
23216,M010040RH,M010040RH_0003,697.666667,4.000000,475.333333,2.666667,67.9,82.9
23217,M010040RH,M010040RH_0004,720.666667,4.000000,532.000000,3.000000,82.9,97.9
23218,M010040RH,M010040RH_0005,707.000000,4.000000,492.333333,3.000000,97.9,112.9
23219,M010040RH,M010040RH_0006,757.666667,4.333333,536.000000,3.000000,112.9,127.9
23220,M010040RH,M010040RH_0007,733.333333,4.333333,439.000000,2.666667,127.9,142.9
23221,M010040RH,M010040RH_0008,439.333333,2.666667,428.333333,2.666667,142.9,157.9
23222,M010040RH,M010040RH_0009,437.333333,3.000000,424.666667,2.666667,157.9,172.9
23223,M010040RH,M010040RH_0010,482.666667,2.666667,383.666667,2.666667,172.9,187.9


## Emotions

In [22]:
emotions_df = anno_dfs['emotions.tab']

In [23]:
# drop emotion == 'none'
emotions_df = emotions_df[(emotions_df['emotion'] != 'none') & (emotions_df['emotion'] != 'noann')]

In [24]:
# split emotion string and explode df
emotions_df['emotion'] = emotions_df['emotion'].apply(lambda x: x.split(','))
emotions_df = emotions_df.explode(column='emotion')

/tmp/ipykernel_4015467/2432850478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emotions_df['emotion'] = emotions_df['emotion'].apply(lambda x: x.split(','))


In [25]:
# drop user_id
emotions_df.drop(columns=['user_id', 'multi_speaker'], inplace=True)

In [26]:
# group by file_id, segment_id, create a set of emotions
emotions_df = emotions_df.groupby(['file_id', 'segment_id'], as_index=False).agg(set)

In [27]:
# convert set to ordered list
emotions_df['emotion'] = emotions_df['emotion'].apply(lambda x: ', '.join(sorted(list(x))))

In [28]:
emotions_df = emotions_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

In [29]:
emotions_df = emotions_df[emotions_df['file_id'] == video['file_uid']]

In [30]:
emotions_df

,file_id,segment_id,emotion,start,end
15565,M010040RH,M010040RH_0001,"anticipation, joy, sadness",37.9,52.9
15566,M010040RH,M010040RH_0002,"anticipation, disgust, joy, trust",52.9,67.9
15567,M010040RH,M010040RH_0003,"anticipation, joy, sadness",67.9,82.9
15568,M010040RH,M010040RH_0004,"joy, sadness, surprise",82.9,97.9
15569,M010040RH,M010040RH_0005,"anticipation, joy, trust",97.9,112.9
15570,M010040RH,M010040RH_0006,"anticipation, joy, surprise, trust",112.9,127.9
15571,M010040RH,M010040RH_0007,"joy, trust",127.9,142.9
15572,M010040RH,M010040RH_0008,"joy, sadness, trust",142.9,157.9
15573,M010040RH,M010040RH_0009,"joy, sadness, surprise, trust",157.9,172.9
15574,M010040RH,M010040RH_0010,"anger, joy, sadness, surprise, trust",172.9,187.9


## Norms

In [31]:
norms_df = anno_dfs['norms.tab']

In [32]:
# norms map
norms_map = {101: 'doing apology',
102: 'doing criticism',
103: 'doing greeting',
104: 'doing request',
105: 'doing persuasion',
106: 'doing thanks',
107: 'doing taking leave'}

In [33]:
norms_df = norms_df[(norms_df['norm'] != 'none') & (norms_df['norm'] != 'noann')]

In [34]:
norms_df.drop(columns='user_id', inplace=True)

/tmp/ipykernel_4015467/3182427442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norms_df.drop(columns='user_id', inplace=True)


In [35]:
norms_df = norms_df.groupby(['file_id', 'segment_id'], as_index=False).agg(list)

In [36]:
norms_df = norms_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

In [37]:
norms_df = norms_df[norms_df['file_id'] == video['file_uid']]

In [38]:
# pull in norms map
norms_df['norm'] = norms_df['norm'].apply(lambda x: [norms_map[int(y)] for y in x])

In [39]:
norms_df

,file_id,segment_id,norm,status,start,end
2528,M010040RH,M010040RH_0003,[doing request],[adhere],67.9,82.9
2529,M010040RH,M010040RH_0007,"[doing thanks, doing taking leave]","[adhere, adhere]",127.9,142.9
2530,M010040RH,M010040RH_0015,[doing thanks],[adhere],247.9,262.9


## Changepoint

In [40]:
changepoint_df = anno_dfs['changepoint.tab']

In [41]:
changepoint_df.drop(columns='user_id', inplace=True)

In [42]:
changepoint_df = changepoint_df[changepoint_df['file_id'] == video['file_uid']]

In [43]:
changepoint_df

,file_id,timestamp,impact_scalar,comment
725,M010040RH,171.0,2,Pre-change: The content creator is upset that ...


## Create JSON file containing all annotations

In [44]:
video_annos = {}
video_annos['valence_arousal'] = valence_df.to_dict('records')
video_annos['emotion'] = emotions_df.to_dict('records')
video_annos['norms'] = norms_df.to_dict('records')
video_annos['changepoint'] = changepoint_df.to_dict('records')

In [45]:
video_annos['changepoint']

[{'file_id': 'M010040RH',
  'timestamp': 171.0,
  'impact_scalar': 2,
  'comment': "Pre-change: The content creator is upset that his roommates are leaving and is laying it on thick. Shift: The friend doesn't play along, asking if he is terminally ill. The mood changed from lighthearted to awkward. Evidence: awkward facial expression"}]

In [46]:
with open(f'{video["file_uid"]}_annotations.json', 'w') as f:
    simplejson.dump(video_annos, f)

### Retrieve predictions for video

In [47]:
cd = 'CCU_P1_TA1_CD_COL_LDC2022E22-V1_20221128_150559'
ad = 'CCU_P1_TA1_AD_COL_LDC2022E22-V1_20221123_141038'
vd = 'CCU_P1_TA1_VD_COL_LDC2022E22-V1_20221123_144304'
ed = 'CCU_P1_TA1_ED_COL_LDC2022E22-V1_20221128_150401'
nd = 'CCU_P1_TA1_ND_COL_LDC2022E22-V1_20221129_101234'
# load predictions for chosen file
submissions_dir = '/home/iron-man/Documents/data/charm/transformed/predictions/submissions'
prediction_dfs = {}
for pred_dir, pred_type in [(cd, 'cd'), (ad, 'ad'), (vd, 'vd'), (ed, 'ed'), (nd, 'nd')]:
    pred_file = os.path.join(submissions_dir, pred_dir, f"{video['file_uid']}.tab")
    prediction_dfs[pred_type] = pd.read_csv(pred_file, delimiter='\t')

In [48]:
# merge valence and arousal
# NB: this won't always work if timestamps aren't aligned
vd_ad_preds_df = prediction_dfs['ad'].merge(prediction_dfs['vd'], how='inner', on=['file_id', 'start', 'end'])

In [49]:
# group ['file_id', 'start', 'end'] and aggregrate to a list
nd_preds_df = prediction_dfs['nd'].groupby(['file_id', 'start', 'end'], as_index=False).agg(list)

In [50]:
temp_df = prediction_dfs['ed'].groupby(['file_id', 'start', 'end'], as_index=False).agg(list)

In [51]:
# check if any intervals have multiple emotions
(temp_df['emotion'].apply(lambda x: len(x)) > 1).sum()

0

In [52]:
# pull in norms map
nd_preds_df['norm'] = nd_preds_df['norm'].apply(lambda x: [norms_map[int(y)] for y in x])

In [53]:
video_preds = {}
video_preds['valence_arousal'] = vd_ad_preds_df.to_dict('records')
video_preds['emotion'] = prediction_dfs['ed'].to_dict('records')
video_preds['norms'] = nd_preds_df.to_dict('records')
video_preds['changepoint'] = prediction_dfs['cd'].to_dict('records')

In [54]:
with open(f'{video["file_uid"]}_predictions.json', 'w') as f:
    simplejson.dump(video_preds, f)